In [10]:
import itertools
import re
import numpy as np
import pandas as pd
import openai
import pandas as pd
import itertools
import streamlit as st
import pickle
# Anna's imports
from transformers import FlaxAutoModelForSeq2SeqLM
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from gradio_client import Client
# from transformers import pipeline

In [26]:
def combinations_of_two(ingredients_input): ###dealt with the issue of missing space crash

    '''
    The function generates all unique pairs of ingredients that can be made from the input list of ingredients.
    NOTE FOR FRONT-END: The output of this function is the input for data_query()
    '''

    ingredients_combinations = []
    powerpowerset = []
    ingredients = re.split(r',', ingredients_input.strip())
    ingredients_list = list(set(ingredient.strip() for ingredient in ingredients))
    for r in range(len(ingredients_list)+1):
        combinations = itertools.combinations(ingredients_list, r)
        #powerset.extend(subset for subset in combinations if len(subset) > 1)
        for comb in combinations:
            if len(comb) > 1:
                if len(comb) < 3:
                    ingredients_combinations.append(comb)
                else:
                    powerpowerset.append(comb)
                    for power in powerpowerset:
                        lowerset = []
                        combins = itertools.combinations(power, 2)
                        for arrange in combins:
                            lowerset.append(arrange)
                    ingredients_combinations.append(lowerset)
    return ingredients_combinations

combinations = combinations_of_two('chicken, onion, garlic, cheese')
combinations

[('chicken', 'garlic'),
 ('chicken', 'cheese'),
 ('chicken', 'onion'),
 ('garlic', 'cheese'),
 ('garlic', 'onion'),
 ('cheese', 'onion'),
 [('chicken', 'garlic'), ('chicken', 'cheese'), ('garlic', 'cheese')],
 [('chicken', 'garlic'), ('chicken', 'onion'), ('garlic', 'onion')],
 [('chicken', 'cheese'), ('chicken', 'onion'), ('cheese', 'onion')],
 [('garlic', 'cheese'), ('garlic', 'onion'), ('cheese', 'onion')],
 [('chicken', 'garlic'),
  ('chicken', 'cheese'),
  ('chicken', 'onion'),
  ('garlic', 'cheese'),
  ('garlic', 'onion'),
  ('cheese', 'onion')]]

In [13]:
df = pd.read_parquet('Halved-DF.parquet.gzip')
df

,ingredient1,ingredient2,scaled_col
0,butter,honey,1.036100
1,butter,mexican seasoning,1.000022
2,butter,mixed spice,1.001687
3,butter,olive oil,1.084810
4,butter,salt,1.553463
...,...,...,...
1335360,nepitella,parmigiano,2.799983
1335361,nepitella,porcini mushrooms,2.799983
1335362,nepitella,salt and fresh pepper,2.799983
1335363,nepitella,unsalted butter,2.799983


In [24]:
def data_query(df, ingredients_combinations): ##Added a penalty of -5 for pairings that are not in the dataframe
    data = []
    for combination in ingredients_combinations:
        if len(combination) < 3:
            ingredient1, ingredient2 = combination
            query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
            score = df.query(query_str)['scaled_col'].values
            print(score)
            if len(score) > 0:
                data.append({'Combination': combination, 'Score': score})
            else:
                continue

        else:
            scores = []
            for i in combination:
                ingredient1, ingredient2 = i
                query_str = f'(ingredient1 == "{ingredient1}" & ingredient2 == "{ingredient2}") | (ingredient1 == "{ingredient2}" & ingredient2 == "{ingredient1}")'
                score = df.query(query_str)['scaled_col'].values
                if len(score) > 0:
                    scores.append(score[0])
                else:
                    scores.append(-5)
            data.append({'Combination': combination, 'Score': scores})
    print(data)
    df_comb = pd.DataFrame(data)
    return df_comb

df_comb = data_query(df, combinations)
df_comb

[1.18008113]
[1.01775019]
[1.32487966]
[1.0944496]
[1.42354091]
[1.41200389]
[{'Combination': ('chicken', 'garlic'), 'Score': array([1.18008113])}, {'Combination': ('chicken', 'cheese'), 'Score': array([1.01775019])}, {'Combination': ('chicken', 'onion'), 'Score': array([1.32487966])}, {'Combination': ('garlic', 'cheese'), 'Score': array([1.0944496])}, {'Combination': ('garlic', 'onion'), 'Score': array([1.42354091])}, {'Combination': ('cheese', 'onion'), 'Score': array([1.41200389])}, {'Combination': [('chicken', 'garlic'), ('chicken', 'cheese'), ('garlic', 'cheese')], 'Score': [1.1800811339314616, 1.0177501859513405, 1.0944496012124034]}, {'Combination': [('chicken', 'garlic'), ('chicken', 'onion'), ('garlic', 'onion')], 'Score': [1.1800811339314616, 1.3248796627107513, 1.4235409132992736]}, {'Combination': [('chicken', 'cheese'), ('chicken', 'onion'), ('cheese', 'onion')], 'Score': [1.0177501859513405, 1.3248796627107513, 1.4120038875676597]}, {'Combination': [('garlic', 'cheese'), 

,Combination,Score
0,"(chicken, garlic)",[1.1800811339314616]
1,"(chicken, cheese)",[1.0177501859513405]
2,"(chicken, onion)",[1.3248796627107513]
3,"(garlic, cheese)",[1.0944496012124034]
4,"(garlic, onion)",[1.4235409132992736]
5,"(cheese, onion)",[1.4120038875676597]
6,"[(chicken, garlic), (chicken, cheese), (garlic...","[1.1800811339314616, 1.0177501859513405, 1.094..."
7,"[(chicken, garlic), (chicken, onion), (garlic,...","[1.1800811339314616, 1.3248796627107513, 1.423..."
8,"[(chicken, cheese), (chicken, onion), (cheese,...","[1.0177501859513405, 1.3248796627107513, 1.412..."
9,"[(garlic, cheese), (garlic, onion), (cheese, o...","[1.0944496012124034, 1.4235409132992736, 1.412..."


In [15]:
def muse_comb(data_query_df): ###If this takes too long, consider taking the nested calculate_sum(array) outside of the function
    '''
     the function calculates the sum of the "Score" values and returns the three combinations with the largest sums
     OUTPUT: [['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['butter', 'eggs', 'pepper', 'cabbage', 'pork', 'flour', 'sugar'],
                 ['yeast', 'butter', 'eggs', 'pepper', 'cabbage', 'flour', 'sugar']]

     NOTE FOR FRONT-END: The return is a list of lists so access the values by indexing e.g. output[0]

                         The output of this function is the input for the recipe generator

                         We might need a function to convert each lists into strings if
                         the recipe generator doesn't do this automatically.
    '''

    def calculate_sum(array):
        return sum(array)

    def ingredients_to_lists(lists):
        ingredients_list = []
        for i in range(3):
            tmp_list = []
            for x in lists[i]:
                tmp_list.append(x[0])
                tmp_list.append(x[1])
            ingredients_list.append(list(set(tmp_list)))

        return ingredients_list

    for i in range(len(data_query_df)):
        data_query_df["Sum"] = data_query_df["Score"].apply(calculate_sum)

    max_values = data_query_df.nlargest(3, "Sum")

    max_values = max_values["Combination"].reset_index(drop=True)

    ingredients_lists = ingredients_to_lists(max_values)

    return ingredients_lists


In [16]:
lists = muse_comb(df_comb)
lists

[['chicken', 'garlic', 'cheese', 'onion'],
 ['garlic', 'cheese', 'onion'],
 ['chicken', 'garlic', 'onion']]

In [17]:
def recipe_generator(ingredients):
    MODEL_NAME_OR_PATH = "flax-community/t5-recipe-generation"
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH, use_fast=True)
    model = FlaxAutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME_OR_PATH)

    prefix = "items: "
    generation_kwargs = {
        "max_length": 512,
        "min_length": 64,
        "no_repeat_ngram_size": 3,
        "do_sample": True,
        "top_k": 60,
        "top_p": 0.95
    }
    special_tokens = tokenizer.all_special_tokens
    tokens_map = {
        "<sep>": "--",
        "<section>": "\n"
    }
    def skip_special_tokens(text, special_tokens):
        for token in special_tokens:
            text = text.replace(token, "")

        return text

    def target_postprocessing(texts, special_tokens):
        if not isinstance(texts, list):
            texts = [texts]

        new_texts = []
        for text in texts:
            text = skip_special_tokens(text, special_tokens)

            for k, v in tokens_map.items():
                text = text.replace(k, v)

            new_texts.append(text)

        return new_texts

    def generation_function(texts):
        _inputs = texts if isinstance(texts, list) else [texts]
        inputs = [str(prefix) + str(inp for inp in _inputs)]
        inputs = tokenizer(
            inputs,
            max_length=256,
            padding="max_length",
            truncation=True,
            return_tensors="jax"
        )

        input_ids = inputs.input_ids
        attention_mask = inputs.attention_mask

        output_ids = model.generate(
            input_ids=input_ids,
            attention_mask=attention_mask,
            **generation_kwargs
        )
        generated = output_ids.sequences
        generated_recipe = target_postprocessing(
            tokenizer.batch_decode(generated, skip_special_tokens=False),
            special_tokens
        )
        return generated_recipe

    recipes = []
    for ingredient in ingredients:
        recipe = generation_function(ingredient)
        recipes.append(recipe)

    return recipes


In [ ]:
k = recipe_generator(lists)

In [19]:
k

[[' title: savory nutty cookie crust\n ingredients: cookie dough-- generator object recipe_generators.html-- t.f.h.t.v.\n directions: 1. preheat oven to 375 degrees. line a cookie sheet with parchment. using a 2 inch cookie cutter, cut out circles of cookie dough. transfer to the cookie sheet. repeat with the remaining cookie dough to make 8 dozen cookies.-- 2. bake for 8 to 10 minutes, or until golden brown. let cool completely.-- 3. store in an airtight container.'],
 [' title: savory nutty cookie crust\n ingredients: cookie dough-- generator object recipe_generators.html-- t.f.h.t.v.\n directions: 1. preheat oven to 375 degrees. line a cookie sheet with parchment. using a 2 inch cookie cutter, cut out circles of cookie dough. transfer to the cookie sheet. repeat with the remaining cookie dough to make 8 dozen cookies.-- 2. bake for 8 to 10 minutes, or until golden brown. let cool completely.-- 3. store in an airtight container.'],
 [' title: savory nutty cookie crust\n ingredients: 

In [14]:
print(k)

[[' title: recipe s colognese ragu recipe\n ingredients: 1 x 3.5 ounce jar ragu r x cream of potato soup see note-- 1 x 10 ounce can flora r g if possible generator object recipe_generator.dark fruit ragu recipe.getstuffing recipe.generator object recipe.data generator object recipe>.remaining ingredients-- 0x4 ounce packet colognese sauce.powdered tomato cream.to yield 3 cups.products.getgetcook.sugar.com.getgettables.com-- 1 x 1 ounce qts. bolognese sauce.ener g lf.c. colognese sauce.ener g lf.com.to yield 3 cups.remove from heat.if using colognese,combine bolognese sauce and cream.do not add in bolognese sauce.prepare 2 oz. kraft lf.c.t.g.e.-- http //www.gr.gf.e.g.u.n.o. gp.p. lf.g.r.p.'], [' title: recipe s colognese ragu recipe\n ingredients: 1 x 3.5 ounce jar ragu r x cream of potato soup see note-- 1 x 10 ounce can flora r g if possible generator object recipe_generator.html-- 1 x 8 ounce jar ragu r g if possible churned recipe page 224. if you don t have generator object, then 

In [16]:
def convert_to_dictionary(recipes):
    recipe_dicts = []
    for recipe in recipes:
        recipe_dict = {}
        recipe = recipe[0]
        parts = recipe.split('\n')
        for part in parts:
            key, value = part.split(':', 1)
            key = key.strip().lower()
            value = value.strip()

            if key in ['ingredients', 'directions']:
                items = [f"{i+1}. {info.strip().capitalize()}" for i, info in enumerate(value.split("--"))]
                recipe_dict[key] = "\n".join(items)
            else:
                recipe_dict[key] = value
        recipe_dicts.append(recipe_dict)

    return recipe_dicts

recipe_dicts = convert_to_dictionary(k)
print(recipe_dicts)

[{'title': 'recipe s colognese ragu recipe', 'ingredients': '1. 1 x 3.5 ounce jar ragu r x cream of potato soup see note\n2. 1 x 10 ounce can flora r g if possible generator object recipe_generator.dark fruit ragu recipe.getstuffing recipe.generator object recipe.data generator object recipe>.remaining ingredients\n3. 0x4 ounce packet colognese sauce.powdered tomato cream.to yield 3 cups.products.getgetcook.sugar.com.getgettables.com\n4. 1 x 1 ounce qts. bolognese sauce.ener g lf.c. colognese sauce.ener g lf.com.to yield 3 cups.remove from heat.if using colognese,combine bolognese sauce and cream.do not add in bolognese sauce.prepare 2 oz. kraft lf.c.t.g.e.\n5. Http //www.gr.gf.e.g.u.n.o. gp.p. lf.g.r.p.'}, {'title': 'recipe s colognese ragu recipe', 'ingredients': '1. 1 x 3.5 ounce jar ragu r x cream of potato soup see note\n2. 1 x 10 ounce can flora r g if possible generator object recipe_generator.html\n3. 1 x 8 ounce jar ragu r g if possible churned recipe page 224. if you don t ha

In [18]:
def get_model():
    with open("utils/model.pickle", "rb") as f:
        model = pickle.load(f)
    return model

model = get_model()

scores = []
recipe_direction = []

for recipe in recipe_dicts:
    if 'directions' in recipe:
        recipe_direction.append(recipe['directions'])
    else:
        recipe_direction.append("")

for direction in recipe_direction:
    scores.append(model.predict_proba([direction])[0][1])

print(scores)

# #scores = [model.predict_proba([recipe['directions']])[0][1] for recipe in recipe_dicts]
# print(scores)

[0.5, 0.5, 0.5]


In [33]:
# def final_recipes(recipe_dict, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't
#     """
#     This evaluates whether the score of a recipe passes or fails the threshold.
#     If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
#     INPUT: Output of the recipe generator function
#     NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
#                         old version for this function to still work.
#                         optimized_gptrecipe() and scoring_model() must be replaced with the actual functions
#     """
#     final_recipes = {"Title": [], "Ingredients": [], "Instructions": []}
#     threshold = 0.1

#     for i in range(len(recipe_dict)):
#         if scores[i] >= threshold:
#             final_recipes["Title"].append([recipe['title'] for recipe in recipe_dicts][i])
#             final_recipes["Ingredients"].append([recipe["ingredients"] for recipe in recipe_dicts][i])
#             final_recipes["Instructions"].append([recipe["directions"] for recipe in recipe_dicts][i])
#         else:
#             n = 0
#             tmp_recipe = {
#                 "Title":[recipe['title'] for recipe in recipe_dicts][i],
#                 "Ingredients":[recipe["ingredients"] for recipe in recipe_dicts][i],
#                 "Instructions":[recipe["directions"] for recipe in recipe_dicts][i]
#                          }
#             last_recipe = {
#                 "Title":[recipe['title'] for recipe in recipe_dicts][i],
#                 "Ingredients":[recipe["ingredients"] for recipe in recipe_dicts][i],
#                 "Instructions":[recipe["directions"] for recipe in recipe_dicts][i]
#                          }
#             while n < 3:
#                 new_recipe = recipe_generator(tmp_recipe["Ingredients"][0]) ###<=== insert actual recipe generator
#                 new_score = model.predict_proba(new_recipe["Instructions"][0]) ###<=== insert the actual scoring model function here
#                 if new_score >= threshold:
#                     final_recipes["Title"].append(new_recipe["Title"])
#                     final_recipes["Ingredients"].append(new_recipe["Ingredients"])
#                     final_recipes["Instructions"].append(new_recipe["Instructions"])
#                     break  # Exit loop if the new recipe passes the threshold
#                 else:
#                     last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
#                     n += 1
#             else: # Add the last generated recipe if the loop completes without finding a passing recipe
#                 final_recipes["Title"].append(last_recipe["Title"])
#                 final_recipes["Ingredients"].append(last_recipe["Ingredients"])
#                 final_recipes["Instructions"].append(last_recipe["Instructions"])
#                 break  # Exit the outer loop to prevent an unending loop

#     return final_recipes

# main_dict = final_recipes(recipe_dicts, scores, model)
# print(main_dict)

In [19]:
print(recipe_dicts)

[{'title': 'recipe s colognese ragu recipe', 'ingredients': '1. 1 x 3.5 ounce jar ragu r x cream of potato soup see note\n2. 1 x 10 ounce can flora r g if possible generator object recipe_generator.dark fruit ragu recipe.getstuffing recipe.generator object recipe.data generator object recipe>.remaining ingredients\n3. 0x4 ounce packet colognese sauce.powdered tomato cream.to yield 3 cups.products.getgetcook.sugar.com.getgettables.com\n4. 1 x 1 ounce qts. bolognese sauce.ener g lf.c. colognese sauce.ener g lf.com.to yield 3 cups.remove from heat.if using colognese,combine bolognese sauce and cream.do not add in bolognese sauce.prepare 2 oz. kraft lf.c.t.g.e.\n5. Http //www.gr.gf.e.g.u.n.o. gp.p. lf.g.r.p.'}, {'title': 'recipe s colognese ragu recipe', 'ingredients': '1. 1 x 3.5 ounce jar ragu r x cream of potato soup see note\n2. 1 x 10 ounce can flora r g if possible generator object recipe_generator.html\n3. 1 x 8 ounce jar ragu r g if possible churned recipe page 224. if you don t ha

In [20]:
recipe_dicts[0]["title"]

'recipe s colognese ragu recipe'

In [21]:
recipe_dicts[0]["ingredients"]

'1. 1 x 3.5 ounce jar ragu r x cream of potato soup see note\n2. 1 x 10 ounce can flora r g if possible generator object recipe_generator.dark fruit ragu recipe.getstuffing recipe.generator object recipe.data generator object recipe>.remaining ingredients\n3. 0x4 ounce packet colognese sauce.powdered tomato cream.to yield 3 cups.products.getgetcook.sugar.com.getgettables.com\n4. 1 x 1 ounce qts. bolognese sauce.ener g lf.c. colognese sauce.ener g lf.com.to yield 3 cups.remove from heat.if using colognese,combine bolognese sauce and cream.do not add in bolognese sauce.prepare 2 oz. kraft lf.c.t.g.e.\n5. Http //www.gr.gf.e.g.u.n.o. gp.p. lf.g.r.p.'

In [22]:
recipe_dicts[0]["directions"]

KeyError: 'directions'

In [44]:
len(recipe_dicts[0])

3

In [58]:
def final_recipes(recipes, scores, model):  ###<=== Function for evaluating if the score passes the threshold and regenerating if it doesn't
    """
    This evaluates whether the score of a recipe passes or fails the threshold.
    If the recipe doesn't meet the threshold after 3 attempts, the last generated recipe is added.
    INPUT: Output of the recipe generator function
    NOTE FOR FRONT-END: it's important to make sure that the outputs of the new recipe generator are the same as the
                        old version for this function to still work.
                        optimized_gptrecipe() and scoring_model() must be replaced with the actual functions
    """
    final_recipes = {"Title": [], "Ingredients": [], "Instructions": []}
    threshold = 2

    for i in range(len(recipe_dicts[0])):
        if scores[i] >= threshold:
            final_recipes["Title"].append(recipes[i]["title"])
            final_recipes["Ingredients"].append(recipes[i]["ingredients"])
            final_recipes["Instructions"].append(recipes[i]["directions"])
        else:
            n = 0
            tmp_recipe = {
                "Title":recipes[i]["title"],
                "Ingredients":recipes[i]["ingredients"],
                "Instructions":recipes[i]["directions"]
                         }
            last_recipe = {
                "Title":recipes[i]["title"],
                "Ingredients":recipes[i]["ingredients"],
                "Instructions":recipes[i]["directions"]
                         }

            while n < 3:
                new_recipe = recipe_generator(tmp_recipe["Ingredients"]) ###<=== insert actual recipe generator
                print(new_recipe)
                new_score = model.predict_proba(new_recipe["Instructions"][0]) ###<=== insert the actual scoring model function here
                if new_score >= threshold:
                    final_recipes["Title"].append(new_recipe["Title"])
                    final_recipes["Ingredients"].append(new_recipe["Ingredients"])
                    final_recipes["Instructions"].append(new_recipe["Instructions"])
                    break  # Exit loop if the new recipe passes the threshold
                else:
                    last_recipe = new_recipe  # Update tmp_recipe with the new recipe if the threshold isn't met
                    n += 1
            else: # Add the last generated recipe if the loop completes without finding a passing recipe
                final_recipes["Title"].append(last_recipe["Title"])
                final_recipes["Ingredients"].append(last_recipe["Ingredients"])
                final_recipes["Instructions"].append(last_recipe["Instructions"])
                break  # Exit the outer loop to prevent an unending loop

    return final_recipes

In [59]:
main_dict = final_recipes(recipe_dicts, scores, model)
print(main_dict)

In [36]:
def image_generator(recipe):
    client = Client("https://playgroundai-playground-v2-5.hf.space/--replicas/xzpmd/")
    result = client.predict(
            recipe,
            " ",
            False,
            820,
            1024,
            1024,
            3,
            True,
            api_name="/run"
    )
    image_path = result[0][0]['image']
    return image_path

In [37]:
for title in main_dict['Title']:
    print(title)
    img = mpimg.imread(image_generator(title))
    plt.imshow(img)
    plt.axis('off')  # Hide axis'/private/var/folders/6k/47szr19n3q9b475tdmrfp68h0000gn/T/gradio/f9f951589827e2d9fd6fb40fe7113f24e3282141/060ccfb9-1a67-4149-a140-62f9694e17bb.png'
    plt.show()


NameError: name 'main_dict' is not defined